In [6]:
!pip3 install arxiv wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Using cached beautifulsoup4-4.13.3-py3-none-any.whl (186 kB)
  Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
    Running setup.py install for wikipedia ... done
You should consider upgrading via the '/Users/ashoksaidoredla/Documents/Langchain_LLM/Langchain_venv/bin/python3 -m pip install --upgrade pip' command.


In [4]:
## Wikipedia tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [5]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max= 200)
wiki = WikipediaQueryRun(api_wrapper = wiki_wrapper)

In [6]:
wiki.name

'wikipedia'

In [7]:
## Langsmith tool

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20).split_documents(docs)
vectordb = FAISS.from_documents(documents, OllamaEmbeddings())
retriever = vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
/var/folders/3s/x5_n2zsd0_zgqvh1k9d51lxh0000gn/T/ipykernel_5624/955424097.py:11: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb = FAISS.from_documents(documents, OllamaEmbeddings())


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10c970f70>, search_kwargs={})

In [8]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(retriever, "langsmith_search", "Search for any information about Langsmith. For any questions about Lngsmith, you must use this tool.")
retrieval_tool

Tool(name='langsmith_search', description='Search for any information about Langsmith. For any questions about Lngsmith, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10a03cb80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10c970f70>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10a632670>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10c970f70>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, templ

In [9]:
retrieval_tool.name

'langsmith_search'

In [10]:
## Arxiv Tool

from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [11]:
tools  = [wiki, arxiv, retrieval_tool]

In [12]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/ashoksaidoredla/Documents/Langchain_LLM/Langchain_venv/lib/python3.9/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for any information about Langsmith. For any questions about Lngsmith, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10a03cb80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vec

In [18]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model_name = "Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x118495ca0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11849c9d0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [19]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [21]:
#Agent Executor

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [22]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Get started by creating your first prompt.
Iterate on models and prompts using the Playground.
Manage prompts programmatically in your application.
Was this page helpful?You can leave detailed feedback on GitHub.NextQuick StartObservabilityEvalsPrompt EngineeringCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.

Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet Sta

{'input': 'Tell me about Langsmith',
 'output': "I can now respond directly with the text output from the tool call. Here's the response:\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\nGet started by creating your first prompt.\nIterate on models and prompts using the Playground.\nManage prompts programmatically in your application.\nWas this page helpful?You can leave detailed feedback on GitHub.NextQuick StartObservabilityEvalsPrompt EngineeringCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.\n\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback 

In [26]:
agent_executor.invoke({"input":"what is Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen
Invoking: `wikipedia` with `{'query': 'Machine learning#Machine_Learning_as_a_Field'}`


Page: List of datasets for machine-learning research
Summary: These datasets are used in machine learning (ML) research and have been cited in peer-reviewed academic journals. Datasets are an integral
Invoking: `arxiv` with `{'query': 'machine learning datasets'}`


Published: 2023-03-24
Title: An investigation of licensing of datasets for machine learning based on the GQM model
Authors: Junyu Chen, Norihiro Yoshida, Hiroaki Takada
Summary: Dataset licensing is c
Invoking: `langsmith_search` with `{'query': 'Junyu Chen, Norihiro Yoshida, Hiroaki Takada'}`


Get started by adding tracing to your applic

{'input': 'what is Machine Learning',
 'output': "I think I've reached the end of this conversation. Since the Langsmith tool didn't provide any relevant information for the original question about Machine Learning, I'll stop here. Thank you for the conversation!"}

In [28]:
agent_executor.invoke({"input":"what is 1706.03762"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The do
Invoking: `arxiv` with `{'query': 'Vaswani et al. 2017'}`


Published: 2018-08-28
Title: The Importance of Being Recurrent for Modeling Hierarchical Structure
Authors: Ke Tran, Arianna Bisazza, Christof Monz
Summary: Recent work has shown that recurrent neuralI think I can provide a direct response this time.

The paper "Attention Is All You Need" by Vaswani et al. (2017) is often considered a seminal work in the field of natural language processing, introducing the transformer architecture. It's a different paper from "The Importance of Being Recurrent for Modeling Hierarchical Structure" by Tran et al. (2018), which explores the use of recurrent neural networks for hierarchical structure modeling

{'input': 'what is 1706.03762',
 'output': 'I think I can provide a direct response this time.\n\nThe paper "Attention Is All You Need" by Vaswani et al. (2017) is often considered a seminal work in the field of natural language processing, introducing the transformer architecture. It\'s a different paper from "The Importance of Being Recurrent for Modeling Hierarchical Structure" by Tran et al. (2018), which explores the use of recurrent neural networks for hierarchical structure modeling. Both papers are influential in their respective areas, but they address different research questions and topics.'}